<a href="https://colab.research.google.com/github/matchbou/kaggle-study-20190410/blob/master/Plotly%20vs.%20Matplotlib%20for%20choropleth%20maps_Kaggle.pdf-201904151113-noshow-plotly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# https://www.kaggle.com/ievgenvp/plotly-vs-matplotlib-for-choropleth-maps

# Plotly vs. Matplotlib for choropleth maps.

In [1]:
import plotly
plotly.__version__

'2.7.0'

In [32]:
# plotly 2.7.0にダウングレード
#!pip uninstall plotly
#!pip3 install plotly==2.7.0

Uninstalling plotly-3.6.1:
  Would remove:
    /usr/local/etc/jupyter/nbconfig/notebook.d/plotlywidget.json
    /usr/local/lib/python3.6/dist-packages/_plotly_utils/*
    /usr/local/lib/python3.6/dist-packages/plotly-3.6.1.dist-info/*
    /usr/local/lib/python3.6/dist-packages/plotly/*
    /usr/local/lib/python3.6/dist-packages/plotlywidget/*
    /usr/local/share/jupyter/nbextensions/plotlywidget/extension.js
    /usr/local/share/jupyter/nbextensions/plotlywidget/index.js
    /usr/local/share/jupyter/nbextensions/plotlywidget/index.js.map
Proceed (y/n)? y
y
  Successfully uninstalled plotly-3.6.1


In [2]:
import plotly
plotly.__version__

'2.7.0'

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (2).json


{'kaggle.json': b'{"username":"matchbou","key":"fb2093338e9cd630d62e0949908398dc"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [5]:
!pip install kaggle

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [7]:
# kaggleデータ置き場とjupyernoteのカレントディレクトリフォルダを作成

!mkdir ./input
!mkdir ./notebook

mkdir: cannot create directory ‘./input’: File exists
mkdir: cannot create directory ‘./notebook’: File exists


In [0]:
#osライブラリ読込してinputに移動⇒kaggleデータもダウンロード　⇒ notebookフォルダに移動
import os
os.chdir('input')



In [9]:
!pwd
!ls -a

/content/input
 .			     FAF4_Regional.csv
 ..			    'FAF4 User Guide.pdf'
 CFS_AREA_shapefile_010215   freight-analysis-framework.zip
 FAF4.4_State.csv


In [8]:
#https://www.kaggle.com/c/titanic/data
!kaggle datasets download -d usdot/freight-analysis-framework

freight-analysis-framework.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
#!unzip freight-analysis-framework.zip -d ./

In [10]:
!pwd
!ls -a

/content/input
 .			     FAF4_Regional.csv
 ..			    'FAF4 User Guide.pdf'
 CFS_AREA_shapefile_010215   freight-analysis-framework.zip
 FAF4.4_State.csv


In [11]:
#上記操作の結果以下のディレクトリ配置となる

# /content/notebook このipynbのカレントディレクトリ
# /content/input カグルからダウンロードしたcsvファイルの置き場所

!pwd
!ls -a ../input


/content/input
 .			     FAF4_Regional.csv
 ..			    'FAF4 User Guide.pdf'
 CFS_AREA_shapefile_010215   freight-analysis-framework.zip
 FAF4.4_State.csv


In [0]:
#ここからカーネルの内容に踏み込む

In [13]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go

from IPython.display import HTML
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [0]:
states_dict = {'01': "AL", '02': "AK", "04": "AZ", "05": "AR",
              "06": "CA", "08": "CO", "09": "CT", "10": "DE",
              "11": "DC", "12": "FL", "13": "GA", "15": "HI",
              "16": "ID", "17": "IL", "18": "IN", "19": "IA",
              "20": "KS", "21": "KY", "22": "LA", "23": "ME",
              "24": "MD", "25": "MA", "26": "MI", "27": "MN",
              "28": "MS", "29": "MO", "30": "MT", "31": "NE",
              "32": "NV", "33": "NH", "34": "NJ", "35": "NM",
              "36": "NY", "37": "NC", "38": "ND", "39": "OH",
              "40": "OK", "41": "OR", "42": "PA", "44": "RI",
              "45": "SC", "46": "SD", "47": "TN", "48": "TX",
              "49": "UT", "50": "VT", "51": "VA", "53": "WA",
              "54": "WV", "55": "WI", "56": "WY"}

years = ['2012', '2013', '2014','2015','2020',
         '2025', '2030', '2035', '2040', '2045']

In [0]:
states_df = pd.read_csv('../input/FAF4.4_State.csv',
                         dtype = {'dms_orig': str, 'dms_dest': str})


In [0]:
states_df['dms_orig'].replace(states_dict, inplace = True)
states_df['dms_dest'].replace(states_dict, inplace = True)

In [0]:
# domestic origin by value 2012-2045 
dom_origin_df = states_df.loc[pd.isnull(states_df['fr_orig'])]

# domestic destination by value 2012-2045 
dom_dest_df = states_df.loc[pd.isnull(states_df['fr_dest'])]

# dataframe for freight balance (outflow from regoin minus inflow to region)
dom_origin_bal_df = dom_origin_df[['dms_orig','value_2012',
                                'value_2013', 'value_2014',
                                'value_2015', 'value_2020',
                                'value_2025', 'value_2030',
                                'value_2035', 'value_2040',
                                'value_2045']].groupby('dms_orig', as_index = True).sum()

dom_dest_bal_df = dom_dest_df[['dms_dest','value_2012',
                                'value_2013', 'value_2014',
                                'value_2015', 'value_2020',
                                'value_2025', 'value_2030',
                                'value_2035', 'value_2040',
                                'value_2045']].groupby('dms_dest', as_index = True).sum()

dom_dest_bal_df = dom_dest_bal_df.apply(lambda x: x*(-1))

balance_df = dom_origin_bal_df.add(dom_dest_bal_df, fill_value = 0.0)
balance_df.reset_index(inplace = True)
balance_df.columns = ['state']+years

In [0]:
scl = [[0.0, 'rgb(84,39,143)'], [0.1, 'rgb(117,107,177)'], [0.2, 'rgb(158,154,200)'],
       [0.3, 'rgb(188,189,220)'], [0.4, '218,218,235)'], [0.5, 'rgb(240,240,240)'],
       [0.6, 'rgb(255,214,151)'],[0.8, 'rgb(250,195,104)'], [0.9, 'rgb(250,177,58)'],
       [1.0, 'rgb(252,153,6)']]

data_bal = []

data_2012 = [dict(type='choropleth',
                colorscale = scl,
                autocolorscale = False,
                locations = balance_df['state'],
                z = balance_df['2012'].astype(float)/1000,
                locationmode = 'USA-states',
                text = balance_df['state'],
                marker = dict(line = dict(color = 'rgb(255,255,255)',
                                          width = 2)),
                visible = True,
                colorbar = dict(title = "Billions USD"))]
    
data_bal.extend(data_2012)

for i in years[1:]:
    data_upd = [dict(type='choropleth',
                      colorscale = scl,
                      autocolorscale = False,
                      locations = balance_df['state'],
                      z = balance_df[i].astype(float)/1000,
                      locationmode = 'USA-states',
                      text = balance_df['state'],
                      marker = dict(line = dict(color = 'rgb(255,255,255)',
                                                width = 2)),
                      visible = False,
                      colorbar = dict(title = "Billions USD"))]
    
    data_bal.extend(data_upd)

# set menues inside the plot
steps = []
yr = 0


for i in range(0,len(data_bal)):
    step = dict(method = "restyle",
                args = ["visible", [False]*len(data_bal)],
                label = years[yr]) 
    step['args'][1][i] = True
    steps.append(step)
    yr += 1

sliders = [dict(active = 10,
                currentvalue = {"prefix": "Year: "},
                pad = {"t": 50},
                steps = steps)]    

layout = dict(title = 'Production / consumption balance per state',
              geo = dict(scope='usa',
                         projection=dict( type='albers usa' ),
                         showlakes = True,
                         lakecolor = 'rgb(255, 255, 255)'),
              sliders = sliders)


fig = dict(data=data_bal, layout=layout)


In [19]:
iplot( fig, filename='d3-cloropleth-map')

In [25]:
!ls -a ../.config

.   active_config    configurations  .last_update_check.json
..  config_sentinel  gce	     logs
